# Fashion MNIST
Fashion Mnist is a Dataset created by Zolando Fashion Wear to replace the Original Mnist and at the same time increasing the difficulty.
This code post is all about how to create a NN model to classify fashion mnist images.

Let’s look at the code.

# 1. Import the Libraries

In [ ]:
%matplotlib inline

In [ ]:
#importing the libraries
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# 2. Load the Dataset 
Pytorch’s torchvision contains inbuilt datasets for practice, such as MNIST, FashionMnist, etc.

In [ ]:
#load datatset 

train_set = torchvision.datasets.FashionMNIST(root = './data/FashionMNIST', download = True,
                                              train = True, transform = transforms.Compose([transforms.ToTensor(),]))

test_set = torchvision.datasets.FashionMNIST(root = './data/FashionMNIST', download=True,
                                             train=False, transform = transforms.Compose([transforms.ToTensor()]))



list of transformations that needs to be done on the input image as a part of pre-processing can be implemented using transforms.compose



# 3. DataLoader for spliting the dataset
Dataset class in pytorch basically covers the data in a tuple and enables us to access the index of each data. It is necessary to create dataloader class which can be used to shuffle, apply Mini-Batch Gradient Descent and more.

We can also show some figures for visualization.

In [ ]:
# dataloader
train_loader = torch.utils.data.DataLoader(train_set,batch_size=20)

test_loader = torch.utils.data.DataLoader(test_set,batch_size=60000)

# iterate the train data with samples
images, labels = next(iter(train_loader))

#used to create a grid of images
grid = torchvision.utils.make_grid(images,nrow=20)
plt.figure(figsize=(15,15))
plt.imshow(np.transpose(grid,(1,2,0)),cmap='gray')


# 4. Define the Neural Network Model
Here, we define the NeuralNetwork as the same as in Lab session **4.2**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# define the model
model = NeuralNetwork()

Note: we just deinfe the simple network model with linear function, you can implete any other structures based on the pytorch framework.

# 5. Set up hyper-parameter
We set up various hyper-parameters, such as number of epochs , loss function , learning rate and optimizer.

We use the Cross-Entropy loss as loss function and SGD with momentum as the optimizer.



In [ ]:
# define learning rate
learning_rate = 0.005
# define your optimizer with SGD and learning rate
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
# define the loss function
criterion = nn.CrossEntropyLoss()
# set the epoch
epochs = 15

# 6. Train the network
When all things are ready, we can train the network now. 

When feeding dataset into the network , we calculate the loss and use the gradient to update the weights by minimizing the loss function. 
We just simply repeat the process several times.

In [ ]:
# loop over the dataset multiple times
losses = []

for i in range(epochs):
  for j,(images,targets) in enumerate(train_loader):
    
    #making predictions
    y_pred = model(images)
  
    #calculating loss
    loss = criterion(y_pred,targets.reshape(-1))
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  if i>10:
    optimizer.lr = 0.0005
  print(loss)
  losses.append(loss)

print('Complete Training')

# 7 . Making a Prediction for A Sample from Test Set



We have trained the network over the training dataset. 
Now, we can check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network
outputs, and checking it against the ground-truth. If the prediction is
correct, we add the sample to the list of correct predictions.
Hence, we can calculate the accuracy of the model on the test data.

For the details, let us display an image from the test data firstly.


In [ ]:
# make a prediction for a sample randomly chose from the test dataset
# you can run this part repeatedly, see the classification result from the model
import random
rand_no = random.randint(0,10000)
print(rand_no)

x_test,y_test = next(iter(test_loader))

y_pred = (model(x_test).argmax(dim=1))

label_list = "T-shirt/Top Trouser PullOver Dress Coat Sandal Shirt Sneaker Bag AnkleBoot".split()

plt.imshow(x_test[rand_no].reshape(28,28),cmap='gray')

pred = model(x_test[rand_no].reshape(-1,1,28,28)).argmax()

print("This is a/an {}".format(label_list[pred]))

# 8. Model Performance Analysis
Let us look at how the network performs on the whole dataset.

In [ ]:

# compute the accuracy of the model

print("Accuracy is : ",(y_pred.eq(y_test).sum()/10000.).item()*100,"%")


<b>Note:</b> 
We use the line plots to show epochs along the ``x-axis`` and the loss error on the ``y-axis``. 
These plots are sometimes called learning curves. These plots can help to diagnose whether the model has over learned, under learned, or is suitably fit to the training dataset.


# 9. Plots loss error 

We can show the loss information for each epoch. 




In [ ]:
# plot loss figure

lossess = torch.stack(losses, 0)
print(type(losses))
print(type(lossess))

lossess = lossess.detach().numpy()
plt.plot(lossess)
plt.xlabel('epochs')
plt.ylabel('loss')


# Conclusion
The model’s accuracy is found to be 86% on the test set. 
This proves how strong the neural networks are in predicting images. 
This is because of the parameter sharing ability and edge-detection ability of network layers.
